In [ ]:
#Code to backtest momentum trading algorithm for the year 2022, 2023 and 2024

import numpy as np
import yfinance as yf
import pandas as pd

RISK_FREE_RATE = 0.02

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker, start_date, end_date):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock = yf.Ticker(adjusted_ticker)
        stock_data = stock.history(start=start_date, end=end_date)

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * np.sqrt(252)
            sharpe_ratio = (annual_return / 100 - RISK_FREE_RATE) / std_dev
            
            pe_ratio = stock.info.get('trailingPE', None)
            stock_name = stock.info.get('longName', None)

            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            expected_return = stock_data['Daily Return'].mean()
            var_95 = expected_return - 1.96 * std_dev
            stock_data['VAR'] = var_95

            return {
                'Ticker': ticker, 
                'Stock Name': stock_name,
                'Return': annual_return, 
                'Std Dev': std_dev, 
                'Sharpe Ratio': sharpe_ratio, 
                'P/E Ratio': pe_ratio, 
                'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1], 
                'VAR': stock_data['VAR'].iloc[-1]
            }, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

def fetch_future_returns(ticker, start_date, end_date):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start=start_date, end=end_date)
        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            return annual_return
        else:
            return None
    except Exception as e:
        print(f"Skipping {ticker} for future returns due to error: {e}")
        return None

def process_year(year, start_date, end_date, sp500_tickers):
    returns_data = []
    start_year = pd.to_datetime(start_date).year 
    next_year_start_date = pd.to_datetime(start_date).replace(year=start_year + 1) 
    next_year_end_date = pd.to_datetime(end_date).replace(year=start_year + 1)

    for ticker in sp500_tickers:
        result, stock_data = fetch_and_analyze_data(ticker, start_date, end_date)
        if result and stock_data is not None:
            returns_data.append(result)
    
    returns_df = pd.DataFrame(returns_data)
    returns_df['P/E Ratio'] = pd.to_numeric(returns_df['P/E Ratio'], errors='coerce')
    returns_df = returns_df.dropna(subset=['P/E Ratio'])
    returns_df = returns_df[returns_df['P/E Ratio'] < 1000]

    average_pe_ratio = returns_df['P/E Ratio'].mean()
    print(f"Average PE Ratio for {year}: {average_pe_ratio}\n")

    filtered_df = returns_df[(returns_df['P/E Ratio'] < average_pe_ratio) & (returns_df['Momentum Health Indicator'] > 10)]
    filtered_df_sorted = filtered_df.sort_values(by='Return', ascending=False)
    top_10_stocks = filtered_df_sorted.head(10)
    
    top_10_stocks['Next Year Return'] = top_10_stocks['Ticker'].apply(lambda x: fetch_future_returns(x, next_year_start_date.strftime('%Y-%m-%d'), next_year_end_date.strftime('%Y-%m-%d')))
    
    # Round values to 1 decimal place
    top_10_stocks = top_10_stocks.round(1)
    
    # Print the top 10 stocks and their metrics
    print(top_10_stocks)
    print(f"\nAverage Return for Next Year: {top_10_stocks['Next Year Return'].mean():.1f}\n")
    
    # Save to Excel
    top_10_stocks.to_excel('top_10_stocks.xlsx', index=False)
    
    return top_10_stocks

def calculate_dates(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    return start_date, end_date

def main():
    sp500_tickers = get_sp500_tickers()
    year = int(input("Enter the year for analysis: "))
    start_date, end_date = calculate_dates(year)
    top_10_stocks = process_year(year, start_date, end_date, sp500_tickers)

if __name__ == "__main__":
    main()

Enter the year for analysis:  2023
